In [1]:

import importlib
import pickle
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from plotting import plot_tsne_selection_grid
import matplotlib.pyplot as plt
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
import glob
import seaborn as sns

from cellCnn.ms.utils.helpers import *
from cellCnn.ms.utils.helpers import calc_frequencies
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p

from plotting import plot_filters, discriminative_filters


def reload_modules():
    import cellCnn.loss_v2
    import cellCnn.utils
    import cellCnn.plotting
    import cellCnn.loss_history

    importlib.reload(cellCnn.loss_v2)
    importlib.reload(cellCnn.ms.utils.helpers)
    importlib.reload(cellCnn.utils)
    importlib.reload(cellCnn.plotting)
    importlib.reload(cellCnn.model)
    importlib.reload(cellCnn.loss_history)
    importlib.reload(cellCnn)
    importlib.reload(cellCnn.ms.utils.helpers)
    import cellCnn.loss_v2
    import cellCnn.utils
    import cellCnn.plotting
    import cellCnn.loss_history
    pass

In [2]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
rand_seed = 123
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}
np.random.seed(rand_seed)
##### no duplicates in

In [3]:
### this is my STL mode for the MTL version of CellCNN

indir = '../../../v2_stl/stl_cv_filters'
outdir = '../../v2_stl_results'
mkdir_p(outdir)
files = glob.glob('../../../v2_stl/**/results.pkl', recursive=True)

In [10]:
for i, file in enumerate(files):
    results = pickle.load(open(file, 'rb'))
    accs = results['accuracies']
    best_net_idx = np.argsort(accs)[::-1][:5] # best net
    print(f'Best accuracy for model {best_net_idx} with filter {i+2} is {accs[best_net_idx]}')
print('Done')

Best accuracy for model [12  2 10  6 11] with filter 2 is [0.6733166  0.6713239  0.66902739 0.66350323 0.65588808]
Best accuracy for model [10  6  7 12 14] with filter 3 is [0.76747465 0.75969374 0.75831383 0.75543159 0.75479418]
Best accuracy for model [ 6 10  0  2 13] with filter 4 is [0.8553586  0.74618381 0.72800165 0.72234136 0.68726158]
Best accuracy for model [10  5  3  0 12] with filter 5 is [0.77108568 0.76562041 0.75849235 0.75583851 0.73325408]
Best accuracy for model [ 5 10  1  8  0] with filter 6 is [0.79226571 0.73387414 0.70885915 0.70262223 0.67925233]
Best accuracy for model [ 0  5  1 10 12] with filter 7 is [0.74435949 0.74177021 0.70652932 0.69080085 0.64896286]
Best accuracy for model [ 5  1 10 13  6] with filter 8 is [0.79171473 0.73367655 0.70177621 0.68224567 0.66641366]
Best accuracy for model [ 7  5 10  6  1] with filter 9 is [0.78390902 0.72876537 0.72723758 0.70990968 0.70971233]
Best accuracy for model [0 5 1 7 9] with filter 10 is [0.74573159 0.72541255 0.7

In [17]:
indir = '../../../v1_stl/cv_score_test/'
outdir = '../../../v1_stl_results'
# mkdir_p(outdir)
nfilters = 13
mean_scores = dict()
for i in range(2, nfilters):
    files = glob.glob(f'{indir}nfilter_{i}/**/results.pkl', recursive=True)
    print('found files')
    print(files)
    best_nets = []
    for file in files:
        results = pickle.load(open(file, 'rb'))
        accs = results['accuracies']
        best_net_idx = np.argsort(accs)[::-1][:10]
        best_nets.append(accs[best_net_idx])
    flat_best_nets = [item for sublist in best_nets for item in sublist]
    print(f'Filter setting {i} has members: {flat_best_nets}')
    mean_scores[i] = sum(flat_best_nets)/len(flat_best_nets)
    print(f'Filter setting {i} scored {sum(flat_best_nets)/len(flat_best_nets)}')
best_score= sorted(mean_scores.values())[::-1][0]

print(f'Best score has {list(mean_scores.keys())[list(mean_scores.values()).index(best_score)]} with {best_score}')
print('done')

found files
['../../../v1_stl/cv_score_test/nfilter_2/kf_0/results.pkl', '../../../v1_stl/cv_score_test/nfilter_2/kf_1/results.pkl', '../../../v1_stl/cv_score_test/nfilter_2/kf_2/results.pkl']
Filter setting 2 has members: [0.6105714440345764, 0.5545238256454468, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.523809552192688, 0.74871426820755, 0.6952857375144958, 0.666523814201355, 0.6636190414428711, 0.641952395439148, 0.6406190395355225, 0.6151905059814453, 0.5993333458900452, 0.597000002861023, 0.5873333215713501, 0.7635999917984009, 0.7063999772071838, 0.6654999852180481, 0.6642500162124634, 0.6621000170707703, 0.6409000158309937, 0.624750018119812, 0.6175000071525574, 0.5999500155448914, 0.5835000276565552]
Filter setting 2 scored 0.611319774389267
found files
['../../../v1_stl/cv_score_test/nfilter_3/kf_0/results.pkl', '../../../v1_stl/cv_score_test/nfilter_3/kf_1/results.pkl', '../../../v1_st

In [11]:
test = glob.glob(f'{indir}nfilter_3/**/results.pkl', recursive=True)
test

['../../../v1_stl/cv_score_test/nfilter_3/kf_0/results.pkl']

In [17]:
indir = '../../../v1_stl/stl_cv_filters/'
files = glob.glob('../../../v1_stl/**/results.pkl', recursive=True)
### CV step 1
for i, file in enumerate(files):
    results = pickle.load(open(file, 'rb'))
    accs = results['accuracies']
    best_net_idx = np.argsort(accs)[::-1][:3] # best net
    print(f'model {best_net_idx} with filter {i+2} is {accs[best_net_idx]} with mean of {accs[best_net_idx].mean()}')
print('Done')


model [12  5  2] with filter 2 is [0.88800001 0.8432222  0.84155554] with mean of 0.8575925827026367
model [ 0 11  6] with filter 3 is [0.81155556 0.80133331 0.796     ] with mean of 0.8029629588127136
model [ 2 14  3] with filter 4 is [0.82866669 0.79622221 0.77866668] with mean of 0.8011851906776428
model [ 8 12  9] with filter 5 is [0.81633335 0.77111113 0.75155556] with mean of 0.7796666820844015
model [ 2 14  4] with filter 6 is [0.81288886 0.77777779 0.77777779] with mean of 0.7894814809163412
model [12 14  2] with filter 7 is [0.8888889  0.86411113 0.78266668] with mean of 0.8452222347259521
model [8 9 4] with filter 8 is [0.83588886 0.7767778  0.77100003] with mean of 0.7945555647214254
model [ 9 14  8] with filter 9 is [0.78966665 0.78211111 0.77755558] with mean of 0.7831111152966818
model [13  8 11] with filter 10 is [0.88788891 0.88700002 0.87188888] with mean of 0.8822592695554098
model [ 7 13 14] with filter 11 is [0.88233334 0.77777779 0.77766669] with mean of 0.81259260